# Obteniendo los datos estadisticos de todos los archivos

Se calculan todas las variables estadisticas y se almacenan en un dataframe

### Importando librerias

In [1]:
import pandas as pd
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
import math
import numpy as np
import matplotlib.pyplot as plt

### Importar lista con nombres de archivos

*Cambiar el nombre de la lista según el sentimiento a analizar

In [2]:
files = pd.read_csv("Anger.csv") ### MODIFICAR EL NOMBRE

### Se crea un diccionario para almacenar las variables de cada archivo

In [3]:
data_instance = {'M1': [],
                 'M2': [],
                 'M3': [],
                 'M4': [],
                 'C2': [],
                 'C3': [],
                 'C4': [],
                 'Skewness': [],
                 'Kurtosis': [],
                 'Mean': [],
                 'Absolute_mean': [],
                 'Peak_value': [],
                 'SRV': [],
                 'RMS': [],
                 'Crest_factor': [],
                 'Shape_factor': []}

### Ciclo para leer todos los archivos de cada sentimiento

In [4]:
for a in range(int(files.count())):
    name_doc = files._get_value (a, 'Name')
    data = pd.read_csv (name_doc)


    ########################################################################################################################################
    ################# Fourier #################
    raw_data = np.ravel(data.to_numpy())
    n = len(raw_data)
    yf = np.fft.fft(raw_data)       #Get fourier transform
    yfts = np.fft.fftshift(yf)
    yftsa = 1.0/n * np.abs(yfts)        #Remove complex numbers

    # Convert numpy array to DataFrame
    data = pd.DataFrame (yftsa)
    ##########################################################################################################################################


    ################# Get statistical features #################
    m1 = float (stats.moment (data, moment = 1)) #First moment 
    m2 = float (stats.moment (data, moment = 2)) #Second moment
    m3 = float (stats.moment (data, moment = 3))#Third moment
    m4 = float (stats.moment (data, moment = 4)) #Fourth moment
    sc = float (m2 - math.sqrt (pow (m1,2))) #Second cumulant
    tc = float (m3 - (3*m1*m2) - (2*pow (m1,3))) #Third cumulant
    fc = float (m4 + (3*m3*m1) - (3*pow (m2,2)) + (12*m2*pow (m1,2)) - (6*pow (m1,4))) #Fourth cumulant
    skew = float (stats.skew (data)) #Skewness
    kurt = float (stats.kurtosis (data)) #Kurtosis
    mean = float(data.mean())# Mean
    am = abs(float(data.mean())) #Absolute mean
    pv = 0.5*(float(data.max()) - float(data.min())) #Peak value
    da = abs(data)
    dsa = da**2
    srv = float(pow((dsa.sum()/int(dsa.count())),2)) #Square root value
    d2 = data**2
    rms = math.sqrt (float(d2.sum())/int(d2.count())) #Root mean square
    cf = pv/rms #Crest factor
    sf = rms/am #Shape factor

    ################# Append features to a dictionary #################
    data_instance ['M1'].append(m1)
    data_instance ['M2'].append(m2)
    data_instance ['M3'].append(m3)
    data_instance ['M4'].append(m4)
    data_instance ['C2'].append(sc)
    data_instance ['C3'].append(tc)
    data_instance ['C4'].append(fc)
    data_instance ['Skewness'].append(skew)
    data_instance ['Kurtosis'].append(kurt)
    data_instance ['Mean'].append(mean)
    data_instance ['Absolute_mean'].append(am)
    data_instance ['Peak_value'].append(pv)
    data_instance ['SRV'].append(srv)
    data_instance ['RMS' ].append(rms)
    data_instance ['Crest_factor'].append(cf)
    data_instance ['Shape_factor'].append(sf)

    del data

### Se crea un dataframe con todos los datos estadisticos y se guarda en un .csv

In [5]:
################# Create a dataframe from dictionary #################
data_stats = pd.DataFrame(data=data_instance)
data_stats.head()

,M1,M2,M3,M4,C2,C3,C4,Skewness,Kurtosis,Mean,Absolute_mean,Peak_value,SRV,RMS,Crest_factor,Shape_factor
0,0.0,1.608201e-07,1.043337e-09,2.266653e-11,1.608201e-07,1.043337e-09,2.258895e-11,16.177592,873.404051,0.000078,0.000078,0.021119,2.784789e-14,0.000409,51.698904,5.249025
1,0.0,1.591426e-07,9.105055e-10,1.891986e-11,1.591426e-07,9.105055e-10,1.884388e-11,14.341776,744.042240,0.000089,0.000089,0.019947,2.791995e-14,0.000409,48.798767,4.584515
2,0.0,4.150035e-07,2.565091e-09,3.897328e-11,4.150035e-07,2.565091e-09,3.845660e-11,9.594562,223.289002,0.000145,0.000145,0.019948,1.901001e-13,0.000660,30.210820,4.556417
3,0.0,3.976761e-07,2.316140e-09,1.715630e-11,3.976761e-07,2.316140e-09,1.668187e-11,9.235718,105.483789,0.000123,0.000123,0.009297,1.703715e-13,0.000642,14.470978,5.230934
4,0.0,3.477772e-07,2.212104e-09,2.020218e-11,3.477772e-07,2.212104e-09,1.983934e-11,10.785838,164.030616,0.000116,0.000116,0.009224,1.305536e-13,0.000601,15.345155,5.164877


In [6]:
#Normalizacion
scaler = MinMaxScaler()
data_stats = pd.DataFrame (scaler.fit_transform(data_stats),columns=['M1','M2','M3','M4','C2','C3','C4','Skewness','Kurtosis','Mean','Absolute_mean','Peak_value','SRV','RMS','Crest_factor','Shape_factor'])

data_stats.to_csv (r"..\stats\Normalizated_data\Stats_anger_fourier.csv")
data_stats.head()

,M1,M2,M3,M4,C2,C3,C4,Skewness,Kurtosis,Mean,Absolute_mean,Peak_value,SRV,RMS,Crest_factor,Shape_factor
0,0.0,0.114494,0.082454,0.155069,0.114494,0.082454,0.161007,0.651455,0.551143,0.262869,0.262869,0.934683,0.012741,0.320791,0.685290,0.457422
1,0.0,0.113294,0.071955,0.129437,0.113294,0.071955,0.134314,0.511297,0.462214,0.303899,0.303899,0.881038,0.012774,0.321013,0.639190,0.191895
2,0.0,0.296261,0.202739,0.266630,0.296261,0.202739,0.274108,0.148863,0.104228,0.505662,0.505662,0.881081,0.086979,0.532620,0.343723,0.180667
3,0.0,0.283870,0.183061,0.117372,0.283870,0.183061,0.118903,0.121467,0.023244,0.425696,0.425696,0.393480,0.077953,0.517610,0.093528,0.450193
4,0.0,0.248187,0.174837,0.138210,0.248187,0.174837,0.141409,0.239813,0.063492,0.402399,0.402399,0.390133,0.059734,0.482813,0.107423,0.423798
